# Creating a 🌈  from Arrays

You can create a `Rainbow` object from arrays representing wavelength, time, flux, and any other array quantities that have the same shape as one of those first three (see [Basics of 🌈 Objects](../basics)). Here, we show how to construct 🌈s from arrays by creating some (very cartoonish) simulated datasets of time-series spectra. 

In [ ]:
from chromatic import Rainbow, RainbowWithModel, SimulatedRainbow, version
from chromatic import np, plt, u

In [ ]:
version()

## Shortest Example
This example shows, in the fewest code lines possible, how to create a 🌈 by supplying your own custom arrays. It doesn't explain things very carefully though, so please read on to the other examples for a friendlier introduction. In this example, we populate a 🌈 with both data and model, but if you don't plan to use the any model comparison features you can simply skip the `model=` keyword.

In [ ]:
N_wavelengths, N_times = 13, 17
r = RainbowWithModel(
    wavelength=np.linspace(1, 2, N_wavelengths) * u.micron,
    time=np.linspace(-0.1, 0.1, N_times) * u.day,
    flux=np.random.normal(1, 0.01, [N_wavelengths, N_times]),
    uncertainty=np.ones([N_wavelengths, N_times]) * 0.01,
    ok=np.random.uniform(0, 1, [N_wavelengths, N_times]) > 0.1,
    model=np.ones([N_wavelengths, N_times]),
)

That's it! We've created a new 🌈 object just by supplying a few arrays. To make sure it worked, let's make a plot.

In [ ]:
r.plot_with_model();

## Simplest Example
*For the simplest example, we'll make a 🌈 out of `wavelength`, `time`, and `flux`.*

First, let's create an array of wavelengths. We'll use [astropy units](https://docs.astropy.org/en/stable/units/index.html) to specify that the units of wavelength are in micron. Setting the units explicitly helps save us from confusion and ruin later on!

In [ ]:
N_wavelengths = 7
my_neat_wavelengths = np.linspace(0.5, 5, N_wavelengths) * u.micron

In [ ]:
my_neat_wavelengths

Next, let's create some times. Again, we'll give them units of time.

In [ ]:
N_times = 11
my_swell_times = np.linspace(-0.1, 0.1, N_times) * u.day

In [ ]:
my_swell_times

And finally, let's make some fluxes associated with each of these wavelengths and times. In general, you'll want to assemble this array of fluxes out of a series of spectra or a group of light curves, but for this example the flux will just be totally random. The first dimension (row) of this array should correspond to wavelength, and the second (column) to time.

In [ ]:
my_great_fluxes = np.random.normal(1, 0.01, size=(N_wavelengths, N_times))

With those arrays, we can create a 🌈 by feeding them in as keywords to `Rainbow`:

In [ ]:
r = Rainbow(wavelength=my_neat_wavelengths, time=my_swell_times, flux=my_great_fluxes)

Ta-da! Now those wavelengths, times, and fluxes have been connected into one 🌈!

In [ ]:
r

In [ ]:
r.imshow();

## Slightly More Complicated Example

*For a tiny bit more complexity, let's also add uncertainties when defining our 🌈.* 

We'll use the same wavelength and time grids as before, but let's define some uncertainties and fluxes together.

In [ ]:
my_cool_uncertainties = np.ones((N_wavelengths, N_times))
my_cool_uncertainties *= np.linspace(0.01, 0.05, N_wavelengths)[:, np.newaxis]

In [ ]:
my_cool_fluxes = np.random.normal(1, my_cool_uncertainties)

To include the uncertainty values, just add an `uncertainty` keyword:

In [ ]:
r = Rainbow(
    wavelength=my_neat_wavelengths,
    time=my_swell_times,
    flux=my_cool_fluxes,
    uncertainty=my_cool_uncertainties,
)

Huzzah! Now there's a rainbow that has an uncertainty associated with each flux. These uncertainties will be helpful if you want to compare your data to models, or if for downweighting more uncertain points when binning together in time or wavelength.

In [ ]:
fi, ax = plt.subplots(1, 2, figsize=(10, 3))
r.imshow(quantity="uncertainty", ax=ax[0])
r.imshow(ax=ax[1]);

## Most Comprehensive Example

*For completeness, let's also add some more quantities that align with either the wavelengths, the times, or the fluxes.* 

Imagine you have a time series of centroid positions (one for each time), or perhaps you recorded the background flux that was subtracted during spectral extraction (one for each wavelength and time), or you have other quantities that would be useful to keep connected to your time-series spectra. Let's make some of these, as examples:

In [ ]:
my_wobbly_centroids = np.random.normal(5, 0.02, N_times) * u.pixel
my_messy_backgrounds = (
    np.random.normal(10, 0.1, (N_wavelengths, N_times)) * u.photon / u.s
)
my_stellar_spectrum = np.random.uniform(2, 3, N_wavelengths) * u.W / u.m**2

You can populate additional arrays inside a rainbow by providing them as additional keyword arguments. Any names are allowed, except for a few protected keywords (`filepath`, `format`, `wavelike`, `timelike`, `fluxlike`, `metadata`). 

In [ ]:
r = Rainbow(
    wavelength=my_neat_wavelengths,
    time=my_swell_times,
    flux=my_cool_fluxes,
    uncertainty=my_cool_uncertainties,
    centroid=my_wobbly_centroids,
    background=my_messy_backgrounds,
    stellar_spectrum=my_stellar_spectrum,
)

Arrays will be sorted into `wavelike`, `timelike`, and `fluxlike` dictionaries based on their shape. 

In [ ]:
r.wavelike

In [ ]:
r.timelike

In [ ]:
r.fluxlike

Let's also attach a model to our `Rainbow` to unlike some snazzy model plotting features. The `.attach_model()` action can be used to do this.

In [ ]:
m = r.attach_model(model=np.ones_like(my_cool_fluxes))

In [ ]:
type(r)

In [ ]:
type(m)

The `.attach_model()` action generates a new type of object, the `RainbowWithModel`. These objects have more abilities that require comparing data to a model. Below, we can call `m.plot_with_model()` but we wouldn't be able to call `r.plot_with_model()` because `r` doesn't have any model associated with it.

In [ ]:
m.plot_with_model();

You can also add arrays directly to a core dictionary by (a) providing a key and an array with the right shape or (b) setting an attribute with an array that would fit in one of the `timelike`, `wavelike`, or `fluxlike` dictionaries. The latter option will try to guess where an array belongs based on its shape, which should *mostly* work. The following two methods should be identical:

In [ ]:
my_imaginary_temperatures = np.random.normal(77, 0.3, N_times)
r.timelike["detector_temperature"] = my_imaginary_temperatures
r.detector_temperature = my_imaginary_temperatures

## Metadata Example

Except for some protected words (`wavelength`, `time`, `flux`, `uncertainty`, `ok`, `model`, other class method names, and a few others), any other attributes you set for a rainbow object will be stored in the `.metadata` core dictionary, so they can be saved and shared. This can be a nice way to document important human-readable information that's useful for interpretting the data.

In [ ]:
s = SimulatedRainbow()

In [ ]:
s.author = "Zach Berta-Thompson"
s.warning = "Watch out! These data are entirely imaginary!"

You can also edit the `.metadata` dictionary directly.

In [ ]:
s.metadata["and another thing"] = "Be kind!"

When we look at the metadata, you'll notice there are a already few other entries that have been automatically populated. If at all possible, it's probably best to try to avoid overwriting those.

In [ ]:
s.metadata

Wahoo! You've done it! Now you can create a 🌈 from whatever arrays and/or data you have available!